In [1]:
!pip install mediapipe
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 32.2 MB 21 kB/s 
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
import os
path = '/content/drive/MyDrive/SheCodes/Dataset/Videos'
lst = os.listdir(path)
lst = sorted(lst)
print("Length of our dataset is:", len(lst))

Length of our dataset is: 129


In [4]:
import cv2
import mediapipe as mp
# If it is our own drive, we can just implement cv2.imshow(window_name, image),
# but this is not permissible on Google Drive, that's why we need this extra step
from google.colab.patches import cv2_imshow
mp_hands = mp.solutions.hands

vid_frame_num = np.zeros((129,), dtype = np.int8)


In [5]:
vid_idx = 0
for video in lst:
    vid = cv2.VideoCapture(path + '/' + video)
    frame_num = 0
    while True:
        ret, frame = vid.read()
        if ret:
            frame_num += 1
        else:
            break
    vid_frame_num[vid_idx] = frame_num
    vid_idx += 1
    print(str(vid_idx) + ': ' + video + " done")

1: 05724_before.mp4 done
2: 05727_before.mp4 done
3: 05728_before.mp4 done
4: 05729_before.mp4 done
5: 05730_before.mp4 done
6: 05731_before.mp4 done
7: 05732_before.mp4 done
8: 05733_before.mp4 done
9: 05734_before.mp4 done
10: 05735_before.mp4 done
11: 05736_before.mp4 done
12: 05737_before.mp4 done
13: 05739_before.mp4 done
14: 05740_before.mp4 done
15: 05741_before.mp4 done
16: 05742_before.mp4 done
17: 05743_before.mp4 done
18: 05744_before.mp4 done
19: 05746_before.mp4 done
20: 05747_before.mp4 done
21: 05749_before.mp4 done
22: 05750_before.mp4 done
23: 08909_candy.mp4 done
24: 08915_candy.mp4 done
25: 08916_candy.mp4 done
26: 08917_candy.mp4 done
27: 08918_candy.mp4 done
28: 08919_candy.mp4 done
29: 08920_candy.mp4 done
30: 08921_candy.mp4 done
31: 08922_candy.mp4 done
32: 08923_candy.mp4 done
33: 08924_candy.mp4 done
34: 08925_candy.mp4 done
35: 08926_candy.mp4 done
36: 08927_candy.mp4 done
37: 08929_candy.mp4 done
38: 12306_computer.mp4 done
39: 12311_computer.mp4 done
40: 12

In [6]:
vid_frame_num

array([ 47,  87,  26,  52,  31,  45,  86, 108,  81,  64,  12,  64,  51,
        48,  31,  28,  47,  67,  48,  76,  90, 100,  53,  94,  86,  32,
        30,  86,  75,  58,  37,  45,  28,  41,  78,  73,  82,  60,  72,
       101,  81,  43,  57,  43,  53,  45,  40, 102,  64,  62,  64,  72,
        73,  46,  88, 120, 103,  78,  90, 101,  75,  91, 107, 105,  68,
        81,  43,  91,  64,  64,  66,  53,  68,  56,  40,  48,  48,  59,
        29,  69,  76,  96,  67,  71,  81,  71,  93,  89,  40,  88,  64,
        61,  33,  63,  19,  64,  64,  33,  60,  60,  45,  45,  45,  42,
        84,  90,  90,  84, 108,  58,  59,  62,  64,  44,  46,  41,  50,
        52,  41,  42,  62, 102,  87,  89,  89,  89, 122, 126,  96],
      dtype=int8)

In [10]:
def hand_tracking_to_np_array(vid_name, frame_num):
    with mp_hands.Hands(      # Initialise mp_hands.Hands
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
            total = np.zeros((frame_num, 2, 21, 3), dtype = np.float_)    # Create numpy array of dimension [ frame_num X 2 hands X 21 landmarks X 3 coordinates (lmark.x, lmark.y, lmark.z) ]
            vid = cv2.VideoCapture(path + '/' + vid_name)   # Reading the video file
            query = True
            frame_idx = 0   # Start from frame 0
            for fr in range(frame_num):
                ret, frame = vid.read()   # Next time you read the vid file: ret, frame will be the next frame
                if ret:       # If frame exists
                    # cv2 read images in BGR format, however mediapipe hands need to read images in RBG format, so conversion is needed
                    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))     # Extracting results, convert frame from BGR to RGB format
                    res = results.multi_hand_landmarks      # NumPy data structure: [ left hand object, right hand object ] 
                    # Note: if 0 hands, tensor length = 0; if only 1 hand, tensor length = 1; if 2 hands, tensor length = 2
                    # To access first hand landmark, call the function hand_lm = res[0].landmark
                    # Then hand_lm is a numpy array of dimension [ 21 landmarks ]
                    # To access the x, y, z coordinate of each landmark, we need to implement
                    # for lmark in hand_lm: print(lmark.x, lmark.y, lmark.z)
                    # Note: if only 1 hand, res[0] will always output the first hand landmark, whereas res[1] will always be none
                    # res[0] and res[1] does not indicate left or right hand in the scenario of only 1 hand detected
                    so = 0
                    if res == None:     # 0 hands detected
                        total[frame_idx, :, :, :] = np.zeros((2, 21, 3))    # Both hands nan padding
                    # Actually we can make it zero at this stage, no need nan padding
                    elif len(res) == 1:       # 1 hand detected
                        lm = 0
                        hand_lm = res[0].landmark   # Focus on that 1 hand; hand_lm is an array of dimension [ 21 landmarks ]
                        for lmark in hand_lm:   # For each landmark; To access coordinates, we need lmark.x, lmark.y, lmark.z
                            total[frame_idx, 0, lm, :] = np.array([lmark.x, lmark.y, lmark.z])
                            lm += 1
                        total[frame_idx, 1, :, :] = np.zeros((21, 3))   # For the remaining hand, we implement zero padding
                    else:     # 2 hands detected; alternatively you can say elif len(res)==2: This will also give the same thing you want
                        lm = 0
                        hand_lm = res[0].landmark   # Left hand first
                        for lmark in hand_lm:
                            total[frame_idx, 0, lm, :] = np.array([lmark.x, lmark.y, lmark.z])
                            lm += 1 
                        lm = 0
                        hand_lm = res[1].landmark   # Now work on the right hand
                        for lmark in hand_lm:
                            total[frame_idx, 1, lm, :] = np.array([lmark.x, lmark.y, lmark.z])
                            lm += 1
                    frame_idx += 1
                else:     # If no frame exist / after the last frame
                    query = False   # Break while loop
            np.save("/content/drive/MyDrive/SheCodes/Dataset/Extracted_hand_landmark_numpy_array/" + vid_name, total)   # Save numpy file to Google Drive

In [14]:
for i in range(129):
    hand_tracking_to_np_array(lst[i], vid_frame_num[i])
    print(str(i) + ': ' + lst[i] + " converted to numpy array!")

0: 05724_before.mp4 converted to numpy array!
1: 05727_before.mp4 converted to numpy array!
2: 05728_before.mp4 converted to numpy array!
3: 05729_before.mp4 converted to numpy array!
4: 05730_before.mp4 converted to numpy array!
5: 05731_before.mp4 converted to numpy array!
6: 05732_before.mp4 converted to numpy array!
7: 05733_before.mp4 converted to numpy array!
8: 05734_before.mp4 converted to numpy array!
9: 05735_before.mp4 converted to numpy array!
10: 05736_before.mp4 converted to numpy array!
11: 05737_before.mp4 converted to numpy array!
12: 05739_before.mp4 converted to numpy array!
13: 05740_before.mp4 converted to numpy array!
14: 05741_before.mp4 converted to numpy array!
15: 05742_before.mp4 converted to numpy array!
16: 05743_before.mp4 converted to numpy array!
17: 05744_before.mp4 converted to numpy array!
18: 05746_before.mp4 converted to numpy array!
19: 05747_before.mp4 converted to numpy array!
20: 05749_before.mp4 converted to numpy array!
21: 05750_before.mp4 co

In [25]:
name = os.listdir("/content/drive/MyDrive/SheCodes/Dataset/Extracted_hand_landmark_numpy_array")
name = sorted(name)
np_array = []
for video in name:
  temp = np.load("/content/drive/MyDrive/SheCodes/Dataset/Extracted_hand_landmark_numpy_array/" + video)
  np_array.append(temp)

In [26]:
name

['05724_before.mp4.npy',
 '05727_before.mp4.npy',
 '05728_before.mp4.npy',
 '05729_before.mp4.npy',
 '05730_before.mp4.npy',
 '05731_before.mp4.npy',
 '05732_before.mp4.npy',
 '05733_before.mp4.npy',
 '05734_before.mp4.npy',
 '05735_before.mp4.npy',
 '05736_before.mp4.npy',
 '05737_before.mp4.npy',
 '05739_before.mp4.npy',
 '05740_before.mp4.npy',
 '05741_before.mp4.npy',
 '05742_before.mp4.npy',
 '05743_before.mp4.npy',
 '05744_before.mp4.npy',
 '05746_before.mp4.npy',
 '05747_before.mp4.npy',
 '05749_before.mp4.npy',
 '05750_before.mp4.npy',
 '08909_candy.mp4.npy',
 '08915_candy.mp4.npy',
 '08916_candy.mp4.npy',
 '08917_candy.mp4.npy',
 '08918_candy.mp4.npy',
 '08919_candy.mp4.npy',
 '08920_candy.mp4.npy',
 '08921_candy.mp4.npy',
 '08922_candy.mp4.npy',
 '08923_candy.mp4.npy',
 '08924_candy.mp4.npy',
 '08925_candy.mp4.npy',
 '08926_candy.mp4.npy',
 '08927_candy.mp4.npy',
 '08929_candy.mp4.npy',
 '12306_computer.mp4.npy',
 '12311_computer.mp4.npy',
 '12312_computer.mp4.npy',
 '12313_c